# nisarh5toimage Tutorial - completely Python free
---

The [nisarhdf](https://github.com/fastice/nisarhdf) contains several Python classes for reading and working with NISAR products.  This tutorial describes `nisarh5toimage`, which is a command line utility that can:

- Provide a quick summary of what's in the file (bands etc) along with essential meta data (e.g., time, orbit, frame, track, center/lat lon, etc.)
- Pull the individual bands from NISAR hdfs and write them to tiff or binary files.
- Produce and accompanying `.vrt` field that makes it easy to import multiband products into python or GIS packages.
- Downsample products (useful for large hi-res GCOV products).
- Produce quickLook PNG files.

## Https, S3, and Local File Access

There are basically three ways this program can access the NISAR h5 products:
1. As a local file on disk (e.g., /myfilesystem/nisarh5file.h5),   
2. As an https link (e.g., https://some.web.site/nisarh5file.h5), and
3. As a file in an s3 bucket (e.g., s3://mys3.bucket/nisarh5file.h5).

Local files are the easiest in that this is the type of file access that hdf5 was designed for. 

Links (https:// and s3://) are more difficult because they don't allow random access. For https, the current solution used for the code described here, is to stream the whole file to memory and perform the desired operations. This can work well, but can create a lot of network traffic, especially if the file is being accessed repeatedly. If this is the case, consider downloading the file locally and performing operations on the local file. 

The case is similar for the s3 buckets, except the code is able to use the `ros3` driver, which allows partial access. So for example, using the `--info` flag (see below), does not download the full file but only enough to read the whole file. 

To keep track of network downloads, the program prints a summary of the network transfers when it exits.

In these examples, we use the [NISAR sample data sets](https://science.nasa.gov/mission/nisar/sample-data/) because they are available to all. 

## Sample Product Links

The only Python in this notebook is used here, to avoid cluttering examples with lengthy sample products. They can be accessed for command line calls with {}, for example `!nisarh5toimage {RSLClink}`.

In [1]:
RSLClink = 'https://nisar.asf.earthdatacloud.nasa.gov/NISAR-SAMPLE-DATA/RSLC/' \
    'NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001/' \
    'NISAR_L1_PR_RSLC_001_030_A_019_2000_SHNA_A_20081012T060910_20081012T060926_D00402_N_F_J_001.h5'
GCOVlink = 'https://nisar.asf.earthdatacloud.nasa.gov/NISAR-SAMPLE-DATA/GCOV/' \
    'NISAR_L2_PR_GCOV_001_030_A_019_2800_SHNA_A_20081012T060911_20081012T060925_D00404_N_F_J_001/' \
    'NISAR_L2_PR_GCOV_001_030_A_019_2800_SHNA_A_20081012T060911_20081012T060925_D00404_N_F_J_001.h5'
GUNWlink = 'https://nisar.asf.earthdatacloud.nasa.gov/NISAR-SAMPLE-DATA/GUNW/' \
    'NISAR_L2_PR_GUNW_001_030_A_019_002_2000_SH_20081012T060911_20081012T060925_20081127T061000_20081127T061014_D00404_N_F_J_001/' \
    'NISAR_L2_PR_GUNW_001_030_A_019_002_2000_SH_20081012T060911_20081012T060925_20081127T061000_20081127T061014_D00404_N_F_J_001.h5'

RIFGlink = 'https://nisar.asf.earthdatacloud.nasa.gov/NISAR-SAMPLE-DATA/RIFG/' \
    'NISAR_L1_PR_RIFG_001_030_A_019_002_2000_SH_20081012T060911_20081012T060925_20081127T061000_20081127T061014_D00404_N_F_J_001/' \
    'NISAR_L1_PR_RIFG_001_030_A_019_002_2000_SH_20081012T060911_20081012T060925_20081127T061000_20081127T061014_D00404_N_F_J_001.h5'

All of the examples below do not include the `--productFamily` option because by default this information is parsed from the file name. If a product such as *RUNW* has a generic name such as, *myFile.h5*, then the call to `nisarh5toimage` should include `--productFamily RUNW`.

## Getting Help

In [2]:
!nisarh5toimage --help

usage: nisarh5toimage [-h] [--info] [--quickLook]
                      [--productFamily {RSLC,ROFF,RIFG,RUNW,GCOV,GUNW,GOFF}]
                      [--frequency {frequencyA,frequencyB}] [--dB] [--sigma0]
                      [--scaleToPixels] [--wrapped]
                      [--polarization {HH,VV,HV,VH}]
                      [--downsampleFactor {2,4,8}]
                      [--fields FIELDS [FIELDS ...]]
                      [--layers {1,2,3} [{1,2,3} ...]]
                      [--outputFormat {COG,GTiff,binary}]
                      productName [output]

Convert NISAR H5 product to Tiff(s), COG(s) or binary files

positional arguments:
  productName           NISAR h5 product (file path, s3, or https)
  output                Root name for ouput (not required for --info)

options:
  -h, --help            show this help message and exit
  --info                Print summary info only
  --quickLook           Write a quick look PNG file (GCOV )
  --productFamily {RSLC,ROFF,RIFG,R

## Get Summary Information

To print a summary of what's in a file use this command:

In [3]:
!nisarh5toimage {RSLClink} --info


Defaults for RSLC: 
HH
VV
HV
VH
Available fields for None: 

Select Parameters
RSLC.track = 30 
RSLC.referenceOrbit = 0 
RSLC.frame = 19 
RSLC.datetime = 2008-10-12 06:09:12 
RSLC.centerLat = None
RSLC.centerLon = None
RSLC.referenceGranule = None
RSLC.SLCNearRange = 743587.3933110968 
RSLC.SLCFarRange = 772526.7340224093 
RSLC.SLCFirstZeroDopplerTime = 22152.0 
RSLC.SLCLastZeroDopplerTime = 22164.999342105264 
RSLC.LookDirection = right 
RSLC.PassType = ascending 
RSLC.Wavelength = None
RSLC.PRF = 1520.0 
RSLC.epsg = None 
RSLC.SLCRangeSize = 6179 
RSLC.SLCAzimuthSize = 19760 
RSLC.SLCRangePixelSize = 4.68425715625 
RSLC.SLCAzimuthPixelSize = 4.484027431092003 
RSLC.SLCZeroDopplerTimeSpacing = 0.0006578947368421052 
RSLC.SLCNearRange = 743587.3933110968 
RSLC.SLCFirstZeroDopplerTime = 22152.0 
RSLC.SLCIncidenceCenter = None

Data Fields
RSLC.HH.HH*
* Indicates data not loaded into memory.

Run time: 
Load data 0:00:06.044503
Write data 0:00:00.000238
Total 0:00:06.044741

Network tra

Note this command loads entire file in memory when reading from https. With s3 links, you can use the `--ros3` to reduce network traffic and memory usage. This should not be in issue for local files.

## Quicklook Images

**Note all of the examples shown in this section can be run with out quicklooks to produce real data.**

Quicklook, downscaled and rescaled images can be created for all products except offsets. Note the rescaling means that these are 'images' not 'data'. In this example to avoid too large an image, a `downsampleFactor` of 8 is used and the data are converted to dB using `-dB`.



### GCOV Quicklook

First make directory save the output files in.

In [4]:
![ -d output ] || mkdir output

In [5]:
!nisarh5toimage {GCOVlink} output/GCOVQL --quickLook --downsampleFactor 8 --dB

computing DB
/home/jovyan/nisarhdf/nisarhdf/nisarBaseHDF.py:1819: RuntimeWarning: invalid value encountered in cast
  arr = arr.astype(np.uint8)

Run time: 
Load data 0:00:02.526452
Write data 0:00:00.153442
Total 0:00:02.679894

Network traffic: 
Bytes sent: 0.645539 MB
Bytes received: 173.430898 MB


By default this call produced a quicklook for the first like-pol (the default). Note you can use `--info` to see both the default and available fields for any product. The resulting quick look is

![](output/GCOVQL.HHHH.png)

### GUNW Quicklook

Note several products have more than one group of data. For example, the *GUNW* products contain both the *unwrapped* and *unwrapped* products. Only on group of products can be access with a single `nisarh5toimage` call. For GUNW products, the default is the unwrapped phase. All of the bands (`--fields all`) can be processed to create quicklooks with the name `ouputRoot.band.png` with the call:

In [6]:
!nisarh5toimage {GUNWlink} output/GUNWQL --quickLook --fields all

/home/jovyan/nisarhdf/nisarhdf/nisarBaseHDF.py:1819: RuntimeWarning: invalid value encountered in cast
  arr = arr.astype(np.uint8)

Run time: 
Load data 0:00:03.255626
Write data 0:00:00.879906
Total 0:00:04.135532

Network traffic: 
Bytes sent: 0.926283 MB
Bytes received: 266.216467 MB


In [7]:
!ls -l output/GUNWQL.*.png

-rw-r--r-- 1 jovyan users 1177361 Sep 27 00:36 output/GUNWQL.coherenceMagnitude.png
-rw-r--r-- 1 jovyan users   10248 Sep 27 00:36 output/GUNWQL.connectedComponents.png
-rw-r--r-- 1 jovyan users   90676 Sep 27 00:36 output/GUNWQL.ionospherePhaseScreen.png
-rw-r--r-- 1 jovyan users  117130 Sep 27 00:36 output/GUNWQL.ionospherePhaseScreenUncertainty.png
-rw-r--r-- 1 jovyan users  701196 Sep 27 00:36 output/GUNWQL.unwrappedPhase.png
-rw-r--r-- 1 jovyan users  441669 Sep 27 00:33 output/GUNWQL.wrappedInterferogram.png


The phase and correlation quicklooks arw shown below:
 Phase | Correlation |
|-------|-----------|
| ![](output/GUNWQL.unwrappedPhase.png) | ![](output/GUNWQL.coherenceMagnitude.png) |

As mentioned above, some products have multiple groups. To access the **wrapped** phase data, the above call is modified as:

In [8]:
!nisarh5toimage {GUNWlink} output/GUNWQL --wrapped --quickLook --downsampleFactor 8

/opt/conda/lib/python3.12/site-packages/matplotlib/colors.py:3175: RuntimeWarning: invalid value encountered in cast
  i = (h * 6.0).astype(int)
/home/jovyan/nisarhdf/nisarhdf/nisarBaseHDF.py:1781: RuntimeWarning: invalid value encountered in cast
  rgb = (mcolors.hsv_to_rgb(hsv) * 255).astype(np.uint8)

Run time: 
Load data 0:00:03.741967
Write data 0:00:00.228663
Total 0:00:03.970630

Network traffic: 
Bytes sent: 1.274629 MB
Bytes received: 266.42866 MB


In [9]:
!ls -l output/GUNWQL.wrappedInterferogram.png

-rw-r--r-- 1 jovyan users 441669 Sep 27 00:36 output/GUNWQL.wrappedInterferogram.png


The quicklook shows the phase of the interfergram as hue over the it's magnitude as the value in and hsv image.
![](output/GUNWQL.wrappedInterferogram.png)

## GeoTIFF Examples

The default output for `nisarh5toimage` is cloud optimized geotiff `--outputFormat COG`, which is geotiff that is optimized for cloud access and which includes pyramids, which make for fast scroll in packages like QGIS and ARGGIS. If pyramids are not needed, the data can be saved as `--outputFormat GTiff`. Finally, the data can also be written out as binary files (`--outputFormat binary`). In all cases, an accompanying `.vrt` file is written, which allows multiple bands to be treated as a single entity. For example, to read in the all the data from an h5 with `rioxarray` as shown in the [notebook](https://github.com/fastice/nisarhdf/blob/main/Notebooks/GUNWHDFtutorial.ipynb) as well as the other `nisarhdf` notebooks.

## RIFG to Tiff Example

In this example the two data fields, *wrappedInterferogram* and *coherenceMagnitude* are saved to cloud-optimized geotiff. In this example, the `--outputFormat COG` is not actually needed because it is the default. This value could be changed to either `GTiff` or `binary`

In [10]:
!nisarh5toimage {RIFGlink} output/RIFGexample --fields all --outputFormat COG
!ls -l output/RIFGexample.*

ref orbit None
ref orbit None

Run time: 
Load data 0:00:02.359593
Write data 0:00:01.807157
Total 0:00:04.166750

Network traffic: 
Bytes sent: 0.364098 MB
Bytes received: 118.182784 MB
-rw-r--r-- 1 jovyan users 37210560 Sep 27 00:36 output/RIFGexample.coherenceMagnitude.tif
-rw-r--r-- 1 jovyan users     3695 Sep 27 00:36 output/RIFGexample.vrt
-rw-r--r-- 1 jovyan users 83159058 Sep 27 00:36 output/RIFGexample.wrappedInterferogram.tif


In [11]:
!cat output/RIFGexample.vrt

<VRTDataset rasterXSize="2058" rasterYSize="3079">
  <SRS dataAxisToSRSAxisMapping="1,2">PROJCS["WGS 84 / UTM zone 11N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-117],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32611"]]</SRS>
  <GeoTransform>  7.4360786723558849e+05,  1.4052771468750000e+01,  0.0000000000000000e+00,  2.2165746381315788e+04,  0.0000000000000000e+00, -4.6052631578947364e-03</GeoTransform>
  <Metadata>
    <MDI key="bands">['wrappedInterferogram', 'coherenceMagnitude']</MDI>
    <MDI key="ByteOrder">LSB</MDI>
  

The above example, illustrates that two COGs (RIFGexample.coherenceMagnitude.tif and RIFGexample.wrappedInterferogram.tif) were created, along with RIFGexample.vrt, the contents of which are shown above. Typically the vrt contains:
1. The projection information (wkt),
2. Geotransform and other geometric information,
3. Some basic SAR parameters, and
4. Some simple band specific information.

**Note the wkt for the range/Doppler products is not the actual wkt and the file is not strictly speaking a geotiff. The geotransform allows the image coordinates to be decoded in terms of range in meters and azimuth time in seconds.**

## GCOV to Binary Example

In this example, the GCOV data is written to flat binary files described by their `.vrt` file. In this example, the full res was more than was needed, so the data were downsampled by a factor 2, converted to sigma0 using the included *rtcGammaToSigmaFactor* field.

In [12]:
!nisarh5toimage {GCOVlink} output/GCOVBinary --downsampleFactor 2  --dB --fields HHHH numberOfLooks rtcGammaToSigmaFactor --outputFormat binary

computing DB

Run time: 
Load data 0:00:03.112970
Write data 0:00:00.835737
Total 0:00:03.948707

Network traffic: 
Bytes sent: 0.578211 MB
Bytes received: 173.239877 MB


In [13]:
!ls -l output/GCOVBinary*

-rw-r--r-- 1 jovyan users 28263680 Sep 27 00:36 output/GCOVBinary.HHHH
-rw-r--r-- 1 jovyan users 28263680 Sep 27 00:36 output/GCOVBinary.numberOfLooks
-rw-r--r-- 1 jovyan users 28263680 Sep 27 00:36 output/GCOVBinary.rtcGammaToSigmaFactor
-rw-r--r-- 1 jovyan users     3365 Sep 27 00:36 output/GCOVBinary.vrt


As with the tiff examples, the `.vrt` file can be used to read the products with packages like rioxarray or imported directly into QGIS.